<div style="text-align: center; font-weight: bold; font-size: 200%"> Negative ions ray neutralization for satellite propulsion</div>                                                        <br />      
<div style="text-align: center; font-size: 150%">Laboratoire de Physique des Plasmas - Ecole polytechnique</div><br />  


<table><tr>
<td> <img src="https://www.lpp.polytechnique.fr/IMG/logo_LPP.png" alt="Drawing" style="width: 30%;"/> </td>
</tr></table>

# DSMC

## Algorithm

For each time step $\Delta t$, 2 phases :
#### Advection phase : 
Particles are moved as if they did not interact. Particles that reach a boundary are processed accordingly.
#### Collision phase : 
For each cell, we compute a given number of collision $M_{coll}$.
1. Choose two particles $i$ and $j$, in the cell, randomly.
2. Compute the collision probability which is proportional to their relative speed and depends on the **maximum relative speed** in the cell which for computational reasons should be estimated.
3. If the pair is accepted, the collision is processed with new velocities depending on the previous one and some randomness.
4. If the pair is rejected, the routine moves again to step 1, until $M_{coll}$ is reached.

## Remarks :
- It is necessary to have enough particles (at least $20$ according to literature) per cell for the validity of the algorithm.
- One of the key point  is estimating the **maximum relative speed** in each cell.
- A simulated particle represents much more physical particles allowing a simulation to take place with roughly $10^4$ particles.

## Principles

- Choose $\Delta t$ so that it is a fraction of the **mean free time**.
- Choose $\Delta x = \Delta y$ (for square cells here) so that is is less that the **mean free path** $\lambda$.

### Calculating $\lambda$ - the mean free path 
By definition : 
$\lambda = \frac{1}{\sqrt{2}{} \pi d^2n}$

With : 
- $d$ : effective diameter of the particule. We choose : $d \approx 4 \times 10^{-10} \space m$ which is the diameter of $I_2$.
- $n$ : density of particules. We choose : $n = [I] \approx 10^{20} \space m^{-3}$.

That yields : $\lambda \approx 0.01 \space m$.

 **Note** : 
- $L \approx 0.005 \space m$.
- We can then compute the **Knudsen** number : $K_n = \frac{\lambda}{L} \approx 2$. It is considered that a continumm description is not accurate when $K_n > 0.1$.

The mean speed of $I$ is roughly $v=3 \times 10^2 \space m.s^{-1}$. Using the mean free path, that yields : $MeanFreeTime \approx 3 \times 10^{-5} \space s$ requiring $\Delta t \leq 3 \times 10^{-6} \space s$.

# Probability of collision

For a given cell containing $N_c$ particles, the number of pairs selected is :

$$M_{cand} = \frac{ \pi N_c \bar{N_c} \sigma^2 v_{r,max} N_e}{2V_c} dt$$ 

Where : 
* $\bar{N_c}$ is, for now, set at the start of the simulation and equals to : $\frac{\text{number of particles}}{\text{number of cells}}$ ;
* $d$ is the effective diameter (here : $4\times10^{-10}$ m) ;
* $v_{r,max}$ is the estimation of the maximum relative norm speed, here $600$ $m.s^{-1}$;
* $N_e$ is the number of real particles a simulated particle represents, here roughly $2 \times 10^{14}$ ;
* $V_c$ is the volume of the cell. 
* $dt$ the time step.

The probability of collision for a given pair $(i,j)$ is then equals to : $\frac{v_{r, norm}}{v_{r,max}}$ where : $v_{r, norm} = \|v_j - v_i\|$.


# Simulation parameters
* $[I] = 10^{20}$ $m^{-3}$
* $ d = 4\times10^{-10}$ $m$
* $v_{max} = 3\times 10^2$ $m.s^{-1}$ and $v_{r,max} = 2v_{max}$

This allows to compute the **mean free time** and the **mean free path**.


* $dt = 0.25 \times MeanFreeTime$
* Mean number of particles per cell : $50$, $100$ or $200$.
* Number of cells : $N_{c,x} = 10$ ; $N_{c,y} = 10$ which makes $100$ cells. The size of the grid is then computed using the **mean free path** $\lambda$: $(\lambda N_{c,x},\lambda N_{c,y}, l_z)$ where $l_z = 0.1$ m. 



## Three types of initializations for speed :
* Uniform  : each $v_x$, $v_y$ and $v_z$ is choosen uniform on $[-m,M]$, $m=M$. The law of  $V = \sqrt{{v_x}^2+{v_y}^2+{v_z}^2}$ can be obtained with geometric arguments (more below).
* Uniform on speed norm : $V = \sqrt{{v_x}^2+{v_y}^2+{v_z}^2}$ is choosen uniformy on $[m,M]$.
* Gaussian : each $v_x$, $v_y$ and $v_z$ is choosen along a gaussian distribution with $\sigma = \sqrt{\frac{k_bT}{m_i}}$ (where $m_i$ is the mass of the particle, here they are all the same) and $\mu = 0$. The distribution of  $V = \sqrt{{v_x}^2+{v_y}^2+{v_z}^2}$ is a maxwellian one.


# Explaination - law of  $V = \sqrt{{v_x}^2+{v_y}^2+{v_z}^2}$


For instance, 
* $\mathbb{P}(V < v) = 0$ for $v<0$.  
* $\mathbb{P}(V < v) = \frac{\text{Volume of the sphere of radius }v \text{ centered in }0}{m^3} = \frac{\frac{4}{3}\pi v^3}{m^3}$ for $0 \leq v \leq m$. 
* $\mathbb{P}(V < v) = ...  $ for $\sqrt{3}m > v > m$.
* $\mathbb{P}(V < v) = 1$ for $\sqrt{3}m < v$.

This yields : $\mathbb{E}(V)=$


##  Modelization of temperature evolution for non maxwellian initializations

We are looking to modelize the standard deviation of the temperature distribution with the relation :
$$\sigma^2(t) = (\sigma^2(0) - \sigma^2_{eq})e^{-t/\tau}+\sigma^2_{eq}$$

Evolution fo temperature in the simulation, for each time step $n$: 
1. We define $w_n = v_n - <v_n>$
2. Which yields : $\sigma^2_n = <w_n^2>$ which is an approximaton of the variance of the random variable $V_n$ by definition.

Since the system is a conservative one, we can compute $<v_{0}^2>$, an approximation of the 2nd order moment of the random variable $V_0$ by definition. Then, if we suppose the thermal equilibrium, we can compute the scale parameter in the Maxwell distribution : $a = \sqrt{\frac{kT}{m}} =  \sqrt{\frac{<v_{0}^2>}{3}}$ for dimension thre. Which yields the variance :  $\sigma_{eq} = \frac{a^2(3\pi-8)}{\pi}$.

Then we minimize $|\sigma^2_n -\big[(\sigma^2(0) - \sigma^2_{eq})e^{-t/\tau}+\sigma^2_{eq}\big]|$ w.r.t. to $\tau$.

## Sources 
* https://mathworld.wolfram.com/MaxwellDistribution.html
* Francis J. Alexander and Alejandro L. Garcia (1997). The Direct Simulation Monte Carlo Method.
* Bird, G. (1994). Molecular Gas Dynamics and the Direct Simulation of Gas Flows.
* https://scicomp.stackexchange.com/questions/19969/how-do-i-generate-maxwell-boltzmann-variates-using-a-uniform-distribution-random


On cherche à modélisation par la relation suivant:

$$T(t) = (T_0 - T_{eq})e^{-t/\tau}+T_{eq}$$

Evolution de la température dans l'expérience, à chaque temps $n$: 
1. On définit $w_n = v_n - <v_n>$
2. Puis on calcule : $<w_n^2>$ qui est une approximation de la variance de la variable aléatoire $V_n$ par définition.
3. Et enfin : $T_{n} = \frac{m}{3k}<w_n^2>$.

Etant donné que le système est conservatif, on calcule $T_{eq} = \frac{m}{3k}<v_{0}^2>$ où $<v_{0}^2>$ est une approximation du moment d'ordre 2 de la variable aléatoire $V_0$ (par définition).

Puis on minimise $|T_{exp,n}-\big[(T_0 - T_{eq})e^{-t/\tau}+T_{eq}\big]|$ par rapport à $\tau$.